## 라이브러리 & API Key 정의

In [15]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import re

load_dotenv() # .env 파일 로드
my_api_key = os.getenv("API_KEY") # 환경 변수에서 API 키 불러오기

client = OpenAI(
    api_key = my_api_key
)

## Instuction 정의

In [2]:
# negative용
instruction_1 = """
You are a recommendation bot responsible for selecting the news article that the target user is most likely to prefer from a list of five candidate articles. The only information available for each candidate article is its title, which is written in Norwegian.

Your goal is to predict the index number of the news article that best fits in the position labeled [MASK].
"""

instruction_negative = """
You are a bot that identifies users' news interests from [News of Interest to the user], then based on this, predicts the index number of news in [Questions] that best fits in the position labeled [MASK].

News is provided by title only.
News is Norwegian news in Norwegian.

There can be multiple lists in [News of Interest to the user], each with five news items.
Among the five news in each list, there is one news that the user is most interested in.

[Questions] can have multiple questions, each of which must be answered.
The answer should return only one news that the user is most likely to read.
"""

# category용
instruction_negative = """
You are a bot that identifies users' news interests from [News of Interest to the user], then based on this, predicts the index number of news in [Questions] that best fits in the position labeled [MASK].

News only provides a title and category.
News is Norwegian news in Norwegian.

There can be multiple lists in [News of Interest to the user], each with five news items.
Among the five news in each list, there is one news that the user is most interested in.

[Questions] can have multiple questions, each of which must be answered.
The answer should return only one news that the user is most likely to read.
"""

# category용 + ndcg
instruction_negative = """
You are a bot that identifies users' news interests from [News of Interest to the user], then based on this, predicts the index number of news in [Questions] that best fits in the position labeled [MASK].

News only provides a title and category.
News is Norwegian news in Norwegian.

There can be multiple lists in [News of Interest to the user], each with five news items.
Among the five news in each list, there is one news that the user is most interested in.

[Questions] can have multiple questions, each of which must be answered.
The answers should sort the index numbers of the news articles in the order that you think best fits the [MASK] based on the user's preferences (only the top 10 will be returned).
"""

# category용 + ndcg2
instruction_negative = """You are a bot that identifies users' news interests from [News of Interest to the user], then based on this, predicts the index number of news in [Questions] that best fits in the position labeled [MASK].

News only provides a title and category.
News is Norwegian news in Norwegian.

There can be multiple lists in [News of Interest to the user], each with five news items.
Among the five news in each list, there is one news that the user is most interested in.

[Questions] can have multiple questions, each of which must be answered. """



instruction_positive = """
You are a bot that identifies users' news interests from [Click History], then based on this, predicts the index number of news in [Questions] that best fits in the position labeled [MASK].

News is provided by title only.
News is Norwegian news in Norwegian.

[Questions] can have multiple questions, each of which must be answered.
The answer should return only one news that the user is most likely to read.
"""

# category용 + ndcg
instruction_positive = """
You are a bot that identifies users' news interests from [Click History], then based on this, predicts the index number of news in [Questions] that best fits in the position labeled [MASK].

News only provides a title and category.
News is Norwegian news in Norwegian.

[Questions] can have multiple questions, each of which must be answered.
The answers should sort the index numbers of the news articles in the order that you think best fits the [MASK] based on the user's preferences (only the top 10 will be returned).
"""

# category용 + ndcg2
instruction_positive = """You are a bot that identifies users' news interests from [Click History], then based on this, predicts the index number of news in [Questions] that best fits in the position labeled [MASK].

News only provides a title and category.
News is Norwegian news in Norwegian.

[Questions] can have multiple questions, each of which must be answered. """

In [3]:
# 0217 ndcg
instruction_negative = """
You are a bot designed to identify users' news interests based on their [News of Interest to the user] and predict the index number of news items in [Questions] that best fit the position labeled [MASK].

Each news article contains only a title and category written in Norwegian.

There can be multiple lists in [News of Interest to the user], each with five news items.
Among the five news in each list, there is one news that the user is most interested in.

[Questions] can have multiple questions, each of which must be answered.
"""


instruction_positive = """
You are a bot designed to identify users' news interests based on their [Click History] and predict the index number of news items in [Questions] that best fit the position labeled [MASK].

Each news article contains only a title and category written in Norwegian.

[Questions] can have multiple questions, each of which must be answered.

<Input example 1>
[Click History]
The news articles that User #15001 clicked before are as follows:
1. Gravid i femte måned - vant NM-finalen [category : ballsport]
2. Trafikkuhell i Vemundvik [category : nordtrondelag]
3. Norgesgruppen kjøper seg inn i grensehandelkjede [category : okonomi]
...

[Questions]
Based on User #15001's preferences, arrange the index numbers of the top five news articles in the sequence that is deemed most suitable for [MASK]

Question 1) User #15001 prefers [MASK] the most among the following five articles: 
1: Silver ber om midlertidig forbud mot offentlig administrasjon [category : okonomi]
2: Perfekt gripefølelse [category : digital]
3: Svenssons debut i Nederland utsettes: Ikke i kveldens Europa League-tropp [category : fotball]
4: Nye tall: Inntil 91 ulver i Norge [category : innenriks]
5: Trygg Trafikk vil ha alkolås i alle biler [category : innenriks]

<Output example 1> 
Question 1: 3, 1, 4, 5, 2

<Input example 2>
[Click History]
The news articles that User #15002 clicked before are as follows:
1. Nå åpner byens nye turløype [category : trondheim]
2. - Jeg har litt erfaring med steinhugging fra før [category : trondheim]
3. «Det mest ettertraktede området i vårt nærområde står i fare for å bli avstengt for allmennheten» [category : ordetfritt]
...

[Questions]
Based on User #15002's preferences, arrange the index numbers of the top five news articles in the sequence that is deemed most suitable for [MASK]

Question 1) User #15002 prefers [MASK] the most among the following five articles: 
1: eAdressa er oppdatert [category : okonomi]
2: Arkitekter reagerer på nytt boligforslag [category : innenriks]
3: Han er en av 13 som får si sitt om fremtidens hær og heimevern [category : nyheter]
4: Hun får halve verden til Trondheim [category : magasin]
5: Kjenner jeg ekstra godt etter så tror jeg at jeg føler meg litt sånn «hin» [category : ordetfritt]

<Output example 2> 
Question 1: 5, 2, 3, 1, 4

<Input example 3>
[Click History]
The news articles that User #15003 clicked before are as follows:
1. Nå åpner byens nye turløype [category : trondheim]
2. Orkangers keiserinne [category : sortrondelag]
3. Mathilde (3) måtte returnere til St. Olavs Hospital etter en dag på Værnes [category : nordtrondelag]
...

[Questions]
Based on User #15003's preferences, arrange the index numbers of the top five news articles in the sequence that is deemed most suitable for [MASK]

Question 1) User #15003 prefers [MASK] the most among the following five articles: 
1: 19-åring tiltalt for gjengvoldtekt i Tromsø [category : innenriks]
2: Innbrudd hos bedrift i Osloveien [category : trondheim]
3: Som vekter i tolv år har jeg fått et ubehagelig innsyn i hva unge spiser i langfriminuttet [category : ordetfritt]
4: Kvinne døde i påkjørsel på E18 ved Horten [category : innenriks]
5: Treneren etter den norske fiaskoen: - Det er nitrist [category : vintersport]

<Output example 3> 
Question 1: 3, 2, 1, 5, 4
"""

In [ ]:
# few-shot


instruction_negative = """
You are a bot designed to identify users' news interests based on their [News of Interest to the user] and predict the index number of news items in [Questions] that best fit the position labeled [MASK].

Each news article contains only a title and category written in Norwegian.

There can be multiple lists in [News of Interest to the user], each with 5 news items.
Among the 5 news in each list, there is one news that the user is most interested in.

[Questions] can have multiple questions, each of which must be answered.

<Q>
[News of Interest to the user]

1) User #10094 prefers most To biler involvert i trafikkulykke ved Støren [category : sortrondelag] among the following 5 articles:
1: To biler involvert i trafikkulykke ved Støren [category : sortrondelag]
2: Sunday Times: Trump vil møte Putin på Island [category : utenriks]
3: Norge vurderer å endre straffestrategi etter bom-bonanza [category : ballsport]
4: Katrine snakker mest med unger og fulle folk [category : magasin]
5: Iraner fikk ett års fengsel etter handletur til Sverige [category : innenriks]

2) User #10094 prefers most «Northug gjør narr av skiledelsen» [category : vintersport] among the following 5 articles:
1: Jensen fikk en halv million av politiet – etter pågripelsen [category : innenriks]
2: Sverresborg er best i klassen [category : nyheter]
3: Jan Guldahl på plass for å hylle Donald Trump [category : utenriks]
4: Dersom du er syk og trenger behandling bør du gå til legen, ikke Snåsamannen [category : ordetfritt]
5: «Northug gjør narr av skiledelsen» [category : vintersport]

3) User #10094 prefers most Toget kjørte på hund [category : nordtrondelag] among the following 5 articles:
1: Jensen fikk en halv million av politiet – etter pågripelsen [category : innenriks]
2: Sverresborg er best i klassen [category : nyheter]
3: Jan Guldahl på plass for å hylle Donald Trump [category : utenriks]
4: Dersom du er syk og trenger behandling bør du gå til legen, ikke Snåsamannen [category : ordetfritt]
5: Toget kjørte på hund [category : nordtrondelag]

4) User #10094 prefers most Jørn kom i stor fart ned skiløypa - så gikk sporet ut på strødd vei [category : trondheim] among the following 5 articles:
1: Tidligere topputøver hevder idrettsstjerner vil boikotte Trump [category : idrettspolitikk]
2: Norge til kvartfinale etter fantastisk omgang: - Ingenting er umulig for denne gjengen [category : ballsport]
3: Jørn kom i stor fart ned skiløypa - så gikk sporet ut på strødd vei [category : trondheim]
4: Over 90 personer pågrepet i Washington [category : utenriks]
5: Politiet fant store mengder partydop nedgravd i Trondheim [category : trondheim]

5) User #10094 prefers most Northug langt bak i sitt første renn på nesten to måneder [category : vintersport] among the following 5 articles:
1: Mener nivået på ishockey i Bergen må heves betraktelig for å fortjene ny arena [category : idrettspolitikk]
2: Northug langt bak i sitt første renn på nesten to måneder [category : vintersport]
3: Nå må vi slutte å bable om trege trøndere [category : meninger]
4: Frivillig overvåking kan gi billigere forsikring [category : innenriks]
5: Knallsterkt comeback av Byåsen i Europacupen [category : ballsport]

6) User #10094 prefers most Mann i 60-årene omkom i ulykke i Overhalla [category : nordtrondelag] among the following 5 articles:
1: Mann i 60-årene omkom i ulykke i Overhalla [category : nordtrondelag]
2: Dette kan gjøres for å unngå overvekt [category : sprek]
3: TV 2 relanserer tippekampen: Sender én Premier League-kamp gratis hver runde [category : fotball]
4: - USA vil oppleve at proteksjonisme har en pris [category : okonomi]
5: Alle de døde funnet etter snøras i Italia [category : utenriks]

7) User #10094 prefers most Boblebad med denne jenta snudde Northugs humør [category : vintersport] among the following 5 articles:
1: Fredriksen og Røkke skaper ny offshore-gigant [category : innenriks]
2: Gro våknet en morgen og var supersvimmel - da ble hun livredd [category : nyheter]
3: Brann-spillerens Facebook-innlegg varmer fansen [category : fotball]
4: Boblebad med denne jenta snudde Northugs humør [category : vintersport]
5: Heltent Jurisic stengte buret til FyllingenBergen [category : ballsport]

[Questions]
Based on User #10094's preferences, predict the index number of the news article that best fits the position labeled [MASK] for each question.

Question 1) User #10094 prefers most [MASK] among the following 5 articles:
1: Flere breiflabber skylles i land i Trøndelag [category : sortrondelag]
2: Veltet vogntog sperret E6 i Grong [category : nordtrondelag]
3: Stol aldri på en trønder - i alle fall ikke når det kommer til mat [category : meninger]
4: Mann skadd i fallulykke i Trondheim sentrum [category : trondheim]
5: Beslagla tyvegods i kjellerbod etter innbrudd på Munkvoll [category : trondheim]

Please provide just the answers to each of User #10094's question without any explanations.

<A>
Question 1 : 2

"""



instruction_positive = """
You are a bot designed to identify users' news interests based on their [Click History] and predict the index number of news items in [Questions] that best fit the position labeled [MASK].

Each news article contains only a title and category written in Norwegian.

[Questions] can have multiple questions, each of which must be answered.

<Q>
[Click History]
The news articles that User #15009 clicked before are as follows:

1. Norges Bank vurderer nye e-penger [category : okonomi]
2. Johaug og Sundby dopinghånet på svensk idrettsfest [category : vintersport]
3. Northugs TV-intervju skapte debatt i skiforbundet [category : vintersport]
4. Her er 17-åringen som forbløffer RBK-sjefen [category : fotball]
5. Slik har det gått for Rema-butikkene i Trøndelag etter den nye strategien [category : okonomi]
6. Trente trøndersk verdensstjerne – nå er han uønsket på livstid [category : andreidretter]
7. Tidligere RBK-talent med unik prestasjon - tidenes første trønder på vei til MLS [category : fotball]
8. Idrettsstjernene frontet nytt kosttilskudd: To år senere er selskapet konkurs [category : andreidretter]
9. Ekspert: – Om du vil ned i vekt, er dette det første jeg anbefaler å kutte ut [category : sprek]
10. Flytrafikken på Værnes sto stille i over fem timer [category : nordtrondelag]
11. Politiet nedringt etter kjempefyrverkeri [category : trondheim]
12. Jakter økseangriper på Klett [category : trondheim]
13. Svensk ekspert: - Johaug kommer ikke til å vinne dette [category : vintersport]
14. Ny lottomillionær i Verdal [category : nordtrondelag]
15. Scoret i første kamp: Dette sier Gytkjær om debuten [category : fotball]
16. Boblebad med denne jenta snudde Northugs humør [category : vintersport]
17. Skimammaens død skaper frykt i smøreboden [category : vintersport]
18. Europatesten går uten Helland for RBK [category : fotball]
19. Trump-kommentaren til Lars fra Tolga er blitt global snakkis [category : innenriks]
20. Derfor er Spalvis-avtalen ennå ikke i boks [category : fotball]
21. Tror tyven kom tilbake om natten og stjal bilen fra garasjen [category : sortrondelag]
22. Det er lett å være tøff i kjeften. Jeg har også slengt med leppa om late arbeidssøkende, men så ble jeg høyst ufrivillig arbeidsledig [category : ordetfritt]
23. Therese Johaug: - Jeg gikk på en smell [category : vintersport]
24. Her får Selnæs fansen til å koke over av «shalala» [category : fotball]
25. RBK fikk Europa midt i fleisen: - Vi fikk sjokk [category : fotball]
26. Gunnes: - Dette vil ikke vare, Sp! [category : innenriks]
27. - Aldri mer samarbeid med Frp [category : innenriks]

[Questions]
Based on User #15009's preferences, predict the index number of the news article that best fits the position labeled [MASK] for each question.

Question 1) User #15009 prefers most [MASK] among the following 5 articles:
1: 43-åring fortsatt drapssiktet etter løslatelse fra varetekt [category : sortrondelag]
2: Tekniske problemer med radio i flere tunneler [category : sortrondelag]
3: Sjansesløsing i årets første vestlandsduell [category : fotball]
4: Fischer villige til å forhandle med Johaug etter endt soning [category : vintersport]
5: Adresseavisens beste bilder fra januar [category : nyheter]

Question 2) User #15009 prefers most [MASK] among the following 5 articles:
1: - Det var så vondt at jeg følte jeg kom til å dø [category : moreromsdal]
2: Kronikk: Trondheim skal være best på buss [category : meninger]
3: Årsaken til at jeg er blitt en slik innbitt DAB-tilhenger [category : ordetfritt]
4: Superbuss: «På langt nær alt vil stå ferdig i 2019» [category : nyheter]
5: Regjeringen foreslår å fjerne krav til drosjeløyve [category : innenriks]

Question 3) User #15009 prefers most [MASK] among the following 5 articles:
1: RBK tjener store penger igjen - sjekk overskuddet [category : fotball]
2: Trøndersk mediekonsern i pluss til tross for sviktende inntekter [category : okonomi]
3: Vil ikke røpe om ny Sluppen bru får statlige midler [category : trondheim]
4: «De enkle svarene i fotballdebatten er feil!» [category : meninger]
5: Fotballpresidenten: Norge gir opp drømmen om å arrangere EM [category : fotball]

<A> 
Question 1: 2
Question 1: 3
Question 1: 1


<Q>
[Click History]
The news articles that User #15013 clicked before are as follows:

1. Full mann fremsto truende med kniv [category : sortrondelag]
2. Stine og samboeren ble frastjålet ting for opp mot 100 000 kroner [category : trondheim]
3. Dette fant politiet etter varsel fra bekymrede naboer [category : trondheim]
4. Dette er RBK-trenerens favoritt til å bli Norges nye landslagstrener [category : fotball]
5. Weng stjal showet som forsanger foran svenskekongen og 25 000 tilskuere [category : vintersport]
6. Mann i 60-årene omkom i ulykke i Overhalla [category : nordtrondelag]
7. Johaug sjekker ut OL-løypene i Sør-Korea [category : vintersport]
8. Her får Selnæs fansen til å koke over av «shalala» [category : fotball]

[Questions]
Based on User #15013's preferences, predict the index number of the news article that best fits the position labeled [MASK] for each question.

Question 1) User #15013 prefers most [MASK] among the following 5 articles:
1: Ølsalget til Rema stuper med ny markedsstrategi [category : okonomi]
2: Skiløperne funnet i god behold [category : sortrondelag]
3: Finsk avis om Bjørndalen: - Kan denne statistikken være sann? [category : vintersport]
4: Tufte hyller 43 år gamle Bjørndalen: – En vanvittig stor inspirasjonskilde [category : vintersport]
5: «Det mest imponerende med Bjørndalen, er hans evige evne til å finne inspirasjon» [category : meninger]

Question 2) User #15013 prefers most [MASK] among the following 5 articles:
1: Rosenborg-treneren: – Vi må luke bort tullefeilene [category : fotball]
2: Vi er ingen skinasjon [category : meninger]
3: Tok førerkortet fredag - tatt for fyllekjøring natt til lørdag [category : nordtrondelag]
4: Mann anmeldt for ruskjøring etter krasj i Leksvik [category : nordtrondelag]
5: Anerkjent advokat kritiserer behandlingen av Johaug: - Bryter reglene [category : vintersport]

Question 3) User #15013 prefers most [MASK] among the following 5 articles:
1: Nå skal Norge sende soldater til Baltikum [category : nyheter]
2: Tyv bet butikkeier [category : sortrondelag]
3: Hvis politikere og byråkrater ikke behersker grunnleggende datasikkerhet, fortjener de ikke skattepengene mine. [category : kommentarer]
4: Bergensk raushet til Trondheim [category : nyheter]
5: - Når noen spør hvem som er pappaen til Sverre, svarer vi «begge» [category : nyheter]

<A> 
Question 1: 3
Question 2: 4
Question 3: 2


<Q>
[Click History]
The news articles that User #15016 clicked before are as follows:

1. - Jeg har litt erfaring med steinhugging fra før [category : trondheim]
2. Mindfullstress i Køben [category : kultur]
3. - Det er en ekstraordinær situasjon som jeg ikke har vært med på før [category : trondheim]
4. Søket etter savnet kvinne er avsluttet [category : trondheim]
5. Tatt etter ulovlig politihacking [category : nyheter]
6. Katrine snakker mest med unger og fulle folk [category : magasin]
7. Therese Johaug: - Jeg gikk på en smell [category : vintersport]
8. Det er nok å lese kommentarene på Farmens Facebook-side for å miste alt håp [category : meninger]
9. Cecilie mener hun har løsningen for dem som «ikke tåler brød» [category : nyheter]
10. Simen var en levende fakkel [category : magasin]

[Questions]
Based on User #15016's preferences, predict the index number of the news article that best fits the position labeled [MASK] for each question.

Question 1) User #15016 prefers most [MASK] among the following 5 articles:
1: Høyre-politiker vil ikke ha rovdyr i Norge [category : innenriks]
2: - Hvitost har vært luksusmat hjemme hos oss [category : nyheter]
3: Ny stiftelse for overgrepsofre [category : innenriks]
4: Sverige og Norge forsterker forsvarssamarbeidet [category : innenriks]
5: Kvinne urinerte på politibil i Trondheim [category : trondheim]

Question 2) User #15016 prefers most [MASK] among the following 5 articles:
1: Dette polet selger 30 prosent mer enn forventet [category : trondheim]
2: Kvinne døde i påkjørsel på E18 ved Horten [category : innenriks]
3: Veltet vogntog sperret E6 i Grong [category : nordtrondelag]
4: Skjerpede Nav-krav kan gi milliardsparing [category : innenriks]
5: Mathallen i Trondheim gir opp [category : innenriks]

Question 3) User #15016 prefers most [MASK] among the following 5 articles:
1: Mange har engasjert seg i rattkjelken til Simon (6) [category : trondheim]
2: Mistenkes for promillekjøring og røyking av hasj [category : trondheim]
3: Beboere på Ranheim vil heller ha trivsel enn tog [category : nyheter]
4: Ansatte blir truet og trakassert av pasienter [category : nyheter]
5: Pro Sentret med varsku om sexbytte blant unge [category : innenriks]

<A> 
Question 1: 2
Question 2: 1
Question 3: 2
"""

In [6]:
# Adressa용
instruction_negative = """
You are a bot designed to identify users' news interests based on their [News of Interest to the user] and predict the index number of news items in [Questions] that best fit the position labeled [MASK].

Each news article contains only a title and category written in Norwegian.

There can be multiple lists in [News of Interest to the user], each with 5 news items.
Among the 5 news in each list, there is one news that the user is most interested in.

[Questions] can have multiple questions, each of which must be answered.
"""



instruction_positive = """
You are a bot designed to identify users' news interests based on their [Click History] and predict the index number of news items in [Questions] that best fit the position labeled [MASK].

Each news article contains only a title and category written in Norwegian.

[Questions] can have multiple questions, each of which must be answered.
"""


instruction_negative_norway = """
Du er en bot designet for å identifisere brukernes nyhetsinteresser basert på deres [Nyheter som brukeren er interessert i] og forutsi indeksnummeret til nyhetsartikler i [Spørsmål] som passer best til posisjonen merket [MASK].

[Nyheter som brukeren er interessert i] kan inneholde flere lister, og hver liste består av 5 nyhetsartikler.
Blant de 5 nyhetene i hver liste er det én nyhet som brukeren er mest interessert i.

[Spørsmål] kan inneholde flere spørsmål, og hvert spørsmål må besvares." 
"""

instruction_positive_norway = """
Du er en bot designet for å identifisere brukernes nyhetsinteresser basert på deres [Klikkhistorikk] og forutsi indeksnummeret til nyhetsartikler i [Spørsmål] som passer best til posisjonen merket [MASK].

[Spørsmål] kan inneholde flere spørsmål, og hvert spørsmål må besvares.
"""

In [2]:
# MIND용
instruction_negative = """
You are a bot designed to identify users' news interests based on their [News of Interest to the user] and predict the index number of news items in [Questions] that best fit the position labeled [MASK].

Each news article contains only a title and category

There can be multiple lists in [News of Interest to the user], each with 5 news items.
Among the 5 news in each list, there is one news that the user is most interested in.

[Questions] can have multiple questions, each of which must be answered.
"""



instruction_positive = """
You are a bot designed to identify users' news interests based on their [Click History] and predict the index number of news items in [Questions] that best fit the position labeled [MASK].

Each news article contains only a title and category

[Questions] can have multiple questions, each of which must be answered.
"""


In [2]:
# MIND summary용
instruction_negative = """
You are a bot designed to identify users' news interests based on their [News Preferences Summary] and predict the index number of news items in [Questions] that best fit the position labeled [MASK].

Each news article contains only a title and category

[Questions] can have multiple questions, each of which must be answered.
"""


instruction_negative = """
You are a bot designed to identify users' news interests based on their [News of Interest to the user] and [News Preferences Summary], and predict the index number of news items in [Questions] that best fit the position labeled [MASK].

Each news article contains only a title

[Questions] can have multiple questions, each of which must be answered.
"""

In [2]:
instruction_negative = """
You are a bot designed to predict which news article best fits the [MASK] position in a [Question], based on a user's [Click History] and [News Preferences Summary].
- Each news article consists of a "title" and a "subcategory" only.
- The [Click History] is a list of news articles the user has previously clicked on, sorted from the earliest to the most recent. Articles with higher index numbers were clicked more recently.
- The [News Preferences Summary] provides a summary of the user's preferred categories or topics.
- The news articles mentioned in the [Question] are more recent than those in the [Click History].
- The [Question] may contain multiple questions. For each question, you must identify the index of the most appropriate news article to fill in the [MASK].

Your task is to accurately predict the index of the news article that best completes each [MASK] in the question(s).
"""

instruction_negative = """
You are a bot designed to predict which news article best fits the [MASK] position in a [Question], based on a user's [News of Interest to the user] and [News Preferences Summary].
- Each news article consists of a "title" and a "subcategory" only.
- The [News of Interest to the user] is a list representing the user's news preferences, sorted from the earliest to the most recent. A higher index number means a more recent preference.
- The [News Preferences Summary] provides a summary of the user's preferred categories or topics.
- The news articles mentioned in the [Question] are more recent than those in the [Click History].
- The [Question] may contain multiple questions. For each question, you must identify the index of the most appropriate news article to fill in the [MASK].

Your task is to accurately predict the index of the news article that best completes each [MASK] in the question(s).
"""

## inference 함수 정의

In [6]:
def inference(purpose, target_folder, result_file_name, gpt_model, user_list, max_attempts):

    # instruction 정의
    # if purpose == 'with_negative': 
    #     instruction = instruction_negative
    # elif purpose == 'only_positive':
    #     instruction = instruction_positive

    if purpose == 'with_negative': 
        instruction = instruction_negative
    elif purpose == 'only_positive':
        instruction = instruction_positive

    # User Prompt가 위치한 폴더 및 metadata 파일 경로 설정
    target_folder = f'../../prompts/{target_folder}'
    directory = f'{target_folder}/{purpose}'
    meta_file_path = f'{target_folder}/{purpose}/metadata/output_metadata.txt'
    user_question_counts = {}
    
    # metadata 파일을 읽어 user별 question 수 저장
    with open(meta_file_path, 'r', encoding='utf-8') as meta_file:
        for line in meta_file:
            match = re.match(r'User ID:\s*U(\d+).*Question 수:\s*(\d+)', line)
            if match:
                user_id = int(match.group(1))
                question_count = int(match.group(2))
                user_question_counts[user_id] = question_count

    # 실험 실행
    with open(f'../../results/gpt_result/{result_file_name}', 'w', encoding='utf-8') as result_file:
        # user list에서 각 user에 대해 처리
        for cnt, i in enumerate(user_list):
            filename = f'U{i}.txt'
            filepath = os.path.join(directory, filename)
            
            # 파일 존재 여부 확인
            if os.path.isfile(filepath):
                # 파일 내용 읽기
                with open(filepath, 'r', encoding='utf-8') as f:
                    contents = f.read()

                # user의 question 수 설정
                expected_question_count = user_question_counts.get(i)
                if expected_question_count is None:
                    print(f"사용자 U{i}의 질문 수를 찾을 수 없습니다.")
                    continue  # 다음 사용자로 넘어감
                
                # API 요청 준비
                initial_messages = [
                    {"role": "system", "content": instruction},
                    {"role": "user", "content": contents}
                ]
                messages = initial_messages.copy()
                attempt = 0

                # 최대 시도 횟수를 넘지 않았으면 실행
                while attempt < max_attempts:
                    attempt += 1
                    # API 호출
                    try:
                        response = client.chat.completions.create(
                            model=gpt_model,
                            messages=messages
                        )
                    except Exception as e:
                        print(f"API 호출 중 오류 발생 (사용자 {i}): {e}")
                        break  # 다음 사용자로 넘어감
                    
                    # 응답 내용 추출
                    response_text = response.choices[0].message.content.strip()
                    
                    
                    result_file.write(f'[U{i}]\n')
                    result_file.write(response_text + '\n\n')
                    if (cnt+1) % 20 == 0:
                        print(f'☆ {purpose} U{i} 까지 완료 [{cnt+1}/{len(user_list)}] ☆')  
                    break  # 루프 종료
                    
            else:
                print(f'파일 {filepath} 이 존재하지 않습니다.')
        print(f'{purpose} 완료 : {result_file_name}\n')


## KCC top1

In [10]:
instruction_positive = """
You are a bot designed to identify users' news interests based on their [Click History] and predict the index number of news items in [Questions] that best fit the position labeled [MASK].

Each news article consists only of a title written in Norwegian.

[Questions] can have multiple questions, each of which must be answered.
"""



user_range = 10000
users = [i for i in range(1, user_range + 1)]

# users = [626]


inference(purpose='only_positive', 
          target_folder='Adressa/KCC/only_title',
          result_file_name='[KCC]Adressa only_title.txt',
          gpt_model='gpt-4o-mini', 
          user_list=users, 
          max_attempts=1
          )

☆ only_positive U20 까지 완료 [20/10000] ☆
☆ only_positive U40 까지 완료 [40/10000] ☆
☆ only_positive U60 까지 완료 [60/10000] ☆
☆ only_positive U80 까지 완료 [80/10000] ☆
☆ only_positive U100 까지 완료 [100/10000] ☆
☆ only_positive U120 까지 완료 [120/10000] ☆
☆ only_positive U140 까지 완료 [140/10000] ☆
☆ only_positive U160 까지 완료 [160/10000] ☆
☆ only_positive U180 까지 완료 [180/10000] ☆
☆ only_positive U200 까지 완료 [200/10000] ☆
☆ only_positive U220 까지 완료 [220/10000] ☆
☆ only_positive U240 까지 완료 [240/10000] ☆
☆ only_positive U260 까지 완료 [260/10000] ☆
☆ only_positive U280 까지 완료 [280/10000] ☆
☆ only_positive U300 까지 완료 [300/10000] ☆
☆ only_positive U320 까지 완료 [320/10000] ☆
☆ only_positive U340 까지 완료 [340/10000] ☆
☆ only_positive U360 까지 완료 [360/10000] ☆
☆ only_positive U380 까지 완료 [380/10000] ☆
☆ only_positive U400 까지 완료 [400/10000] ☆
☆ only_positive U420 까지 완료 [420/10000] ☆
☆ only_positive U440 까지 완료 [440/10000] ☆
☆ only_positive U460 까지 완료 [460/10000] ☆
☆ only_positive U480 까지 완료 [480/10000] ☆
☆ only_positive U500 까지 

In [11]:
instruction_positive = """
You are a bot designed to identify users' news interests based on their [Click History] and predict the index number of news items in [Questions] that best fit the position labeled [MASK].

Each news article consists only of a title and category written in Norwegian.

[Questions] can have multiple questions, each of which must be answered.
"""


user_range = 10000
users = [i for i in range(1, user_range + 1)]

# users = [626]


inference(purpose='only_positive', 
          target_folder='Adressa/KCC/cate',
          result_file_name='[KCC]Adressa cate.txt',
          gpt_model='gpt-4o-mini', 
          user_list=users, 
          max_attempts=1
          )

☆ only_positive U20 까지 완료 [20/10000] ☆
☆ only_positive U40 까지 완료 [40/10000] ☆
☆ only_positive U60 까지 완료 [60/10000] ☆
☆ only_positive U80 까지 완료 [80/10000] ☆
☆ only_positive U100 까지 완료 [100/10000] ☆
☆ only_positive U120 까지 완료 [120/10000] ☆
☆ only_positive U140 까지 완료 [140/10000] ☆
☆ only_positive U160 까지 완료 [160/10000] ☆
☆ only_positive U180 까지 완료 [180/10000] ☆
☆ only_positive U200 까지 완료 [200/10000] ☆
☆ only_positive U220 까지 완료 [220/10000] ☆
☆ only_positive U240 까지 완료 [240/10000] ☆
☆ only_positive U260 까지 완료 [260/10000] ☆
☆ only_positive U280 까지 완료 [280/10000] ☆
☆ only_positive U300 까지 완료 [300/10000] ☆
☆ only_positive U320 까지 완료 [320/10000] ☆
☆ only_positive U340 까지 완료 [340/10000] ☆
☆ only_positive U360 까지 완료 [360/10000] ☆
☆ only_positive U380 까지 완료 [380/10000] ☆
☆ only_positive U400 까지 완료 [400/10000] ☆
☆ only_positive U420 까지 완료 [420/10000] ☆
☆ only_positive U440 까지 완료 [440/10000] ☆
☆ only_positive U460 까지 완료 [460/10000] ☆
☆ only_positive U480 까지 완료 [480/10000] ☆
☆ only_positive U500 까지 

## KCC ranking

In [26]:
instruction_positive = """
You are a bot designed to identify users' news interests based on their [Click History] and predict the index number of news items in [Questions] that best fit the position labeled [MASK].

Each news article consists only of a title written in Norwegian.

[Questions] can have multiple questions, each of which must be answered.
"""



user_range = 10000
users = [i for i in range(1, user_range + 1)]

users = [669, 1005, 1166, 1262, 1469, 1528, 1638, 2199, 2398, 2587, 2923, 2941, 3163, 3209, 3289, 3418, 3958, 4097, 4236, 4388, 4474, 4500, 4718, 4955, 4990, 5008, 5364, 5426, 5479, 5654, 5905, 6894, 6932, 6940, 7127, 7500, 7505, 7579, 7683, 7827, 7941, 7985, 8244, 8518, 8675, 8868, 8951, 9038, 9061, 9069, 9327, 9460, 9510, 9695, 9743, 9781, 9840]


inference(purpose='only_positive', 
          target_folder='Adressa/KCC/ranking/only_title',
          result_file_name='[KCC-ranking]Adressa only_title.txt',
          gpt_model='gpt-4o-mini', 
          user_list=users, 
          max_attempts=1
          )

☆ only_positive U4388 까지 완료 [20/57] ☆
☆ only_positive U7827 까지 완료 [40/57] ☆
only_positive 완료 : [KCC-ranking]Adressa only_title.txt



In [ ]:
instruction_positive = """
You are a bot designed to identify users' news interests based on their [Click History] and predict the index number of news items in [Questions] that best fit the position labeled [MASK].

Each news article consists only of a title and category written in Norwegian.

[Questions] can have multiple questions, each of which must be answered.
"""


user_range = 5450
users = [i for i in range(901, user_range + 1)]

users = [1, 6, 158, 169, 206, 209, 259, 311, 448, 528, 529, 543, 647, 801, 805, 857, 991, 1011, 1072, 1080, 1198, 1208, 1337, 1361, 1411, 1417, 1427, 1485, 1509, 1611, 1797, 1806, 1818, 1870, 1911, 1963, 1996, 2010, 2073, 2165, 2272, 2300, 2356, 2382, 2414, 2428, 2496, 2504, 2547, 2660, 2807, 2839, 2841, 2854, 2862, 2944, 2963, 3005, 3025, 3063, 3147, 3163, 3189, 3197, 3224, 3295, 3336, 3345, 3451, 3534, 3553, 3561, 3619, 3649, 3650, 3754, 3770, 3804, 3810, 3831, 3945, 3965, 3986, 4082, 4093, 4098, 4198, 4267, 4273, 4284, 4334, 4451, 4540, 4547, 4586, 4627, 4678, 4798, 4826, 4842, 4857, 4903, 4944, 4994, 5008, 5020, 5048, 5057, 5164, 5168, 5351, 5447, 5482, 5514, 5549, 5833, 5857, 5870, 5905, 5925, 5994, 6019, 6126, 6139, 6218, 6264, 6308, 6329, 6349, 6367, 6463, 6568, 6637, 6668, 6745, 6751, 6763, 6792, 6812, 6945, 6994, 7097, 7153, 7154, 7244, 7284, 7358, 7366, 7381, 7423, 7441, 7465, 7482, 7503, 7594, 7674, 7749, 7842, 7845, 7954, 8004, 8037, 8156, 8188, 8202, 8223, 8226, 8260, 8341, 8432, 8530, 8551, 8573, 8588, 8605, 8683, 8702, 8725, 8732, 8773, 8787, 8846, 8918, 9052, 9228, 9239, 9245, 9300, 9342, 9368, 9378, 9381, 9455, 9460, 9500, 9519, 9577, 9586, 9675, 9759, 9760, 9839, 9966]


inference(purpose='only_positive', 
          target_folder='Adressa/KCC/ranking/cate',
          result_file_name='[KCC-ranking]Adressa cate.txt',
          gpt_model='gpt-4o-mini', 
          user_list=users, 
          max_attempts=1
          )

☆ only_positive U1080 까지 완료 [20/207] ☆
☆ only_positive U2165 까지 완료 [40/207] ☆
☆ only_positive U3063 까지 완료 [60/207] ☆
☆ only_positive U3810 까지 완료 [80/207] ☆
☆ only_positive U4826 까지 완료 [100/207] ☆
☆ only_positive U5905 까지 완료 [120/207] ☆
☆ only_positive U6812 까지 완료 [140/207] ☆
☆ only_positive U7842 까지 완료 [160/207] ☆
☆ only_positive U8702 까지 완료 [180/207] ☆
☆ only_positive U9577 까지 완료 [200/207] ☆
only_positive 완료 : [KCC-ranking]Adressa cate.txt



---

In [ ]:
instruction_positive = """
You are a bot designed to identify users' news interests based on their [Click History] and predict the index number of news items in [Questions] that best fit the position labeled [MASK].

Each news article contains only a title

[Questions] can have multiple questions, each of which must be answered.
"""

user_range = 1000
users = [i for i in range(1, user_range + 1)]

# users = [62]

# 실행

inference(purpose='only_positive', 
          target_folder='MIND/KCC/only_title',
          result_file_name='[KCC] only_title.txt',
          gpt_model='gpt-4o-mini', 
          user_list=users, 
          max_attempts=1
          )



☆ only_positive U20 까지 완료 [20/1000] ☆
☆ only_positive U40 까지 완료 [40/1000] ☆
☆ only_positive U60 까지 완료 [60/1000] ☆
☆ only_positive U80 까지 완료 [80/1000] ☆
☆ only_positive U100 까지 완료 [100/1000] ☆
☆ only_positive U120 까지 완료 [120/1000] ☆
☆ only_positive U140 까지 완료 [140/1000] ☆
☆ only_positive U160 까지 완료 [160/1000] ☆
☆ only_positive U180 까지 완료 [180/1000] ☆
☆ only_positive U200 까지 완료 [200/1000] ☆
☆ only_positive U220 까지 완료 [220/1000] ☆
☆ only_positive U240 까지 완료 [240/1000] ☆
☆ only_positive U260 까지 완료 [260/1000] ☆
☆ only_positive U280 까지 완료 [280/1000] ☆
☆ only_positive U300 까지 완료 [300/1000] ☆
☆ only_positive U320 까지 완료 [320/1000] ☆
☆ only_positive U340 까지 완료 [340/1000] ☆
☆ only_positive U360 까지 완료 [360/1000] ☆
☆ only_positive U380 까지 완료 [380/1000] ☆
☆ only_positive U400 까지 완료 [400/1000] ☆
☆ only_positive U420 까지 완료 [420/1000] ☆
☆ only_positive U440 까지 완료 [440/1000] ☆
☆ only_positive U460 까지 완료 [460/1000] ☆
☆ only_positive U480 까지 완료 [480/1000] ☆
☆ only_positive U500 까지 완료 [500/1000] ☆
☆ only_p

In [5]:
instruction_negative = """
You are a bot designed to identify users' news interests based on their [News of Interest to the user] and predict the index number of news items in [Questions] that best fit the position labeled [MASK].

Each news article contains only a title 

There can be multiple lists in [News of Interest to the user], each with 5 news items.
Among the 5 news in each list, there is one news that the user is most interested in.

[Questions] can have multiple questions, each of which must be answered.
"""

inference(purpose='with_negative', 
          target_folder='MIND/KCC/negative only_title ns=cate',
          result_file_name='[KCC] negative only_title ns=cate.txt',
          gpt_model='gpt-4o-mini', 
          user_list=users, 
          max_attempts=1
          )

☆ with_negative U20 까지 완료 [20/1000] ☆
☆ with_negative U40 까지 완료 [40/1000] ☆
☆ with_negative U60 까지 완료 [60/1000] ☆
☆ with_negative U80 까지 완료 [80/1000] ☆
☆ with_negative U100 까지 완료 [100/1000] ☆
☆ with_negative U120 까지 완료 [120/1000] ☆
☆ with_negative U140 까지 완료 [140/1000] ☆
☆ with_negative U160 까지 완료 [160/1000] ☆
☆ with_negative U180 까지 완료 [180/1000] ☆
☆ with_negative U200 까지 완료 [200/1000] ☆
☆ with_negative U220 까지 완료 [220/1000] ☆
☆ with_negative U240 까지 완료 [240/1000] ☆
☆ with_negative U260 까지 완료 [260/1000] ☆
☆ with_negative U280 까지 완료 [280/1000] ☆
☆ with_negative U300 까지 완료 [300/1000] ☆
☆ with_negative U320 까지 완료 [320/1000] ☆
☆ with_negative U340 까지 완료 [340/1000] ☆
☆ with_negative U360 까지 완료 [360/1000] ☆
☆ with_negative U380 까지 완료 [380/1000] ☆
☆ with_negative U400 까지 완료 [400/1000] ☆
☆ with_negative U420 까지 완료 [420/1000] ☆
☆ with_negative U440 까지 완료 [440/1000] ☆
☆ with_negative U460 까지 완료 [460/1000] ☆
☆ with_negative U480 까지 완료 [480/1000] ☆
☆ with_negative U500 까지 완료 [500/1000] ☆
☆ with_n